In [1]:
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import dateutil
import analysis

packets = analysis.parse_packet_file('data/all.txt')
packets += analysis.parse_packet_file('data/temp_basals.txt')

# Select valid packets of a particular length
packets = filter(lambda x: x.is_valid() and x.body_len == 10, packets)

# Uncomment if you want to work with less data (faster while developing)
#packets = packets[0:500]

In [2]:
cracked_bits_dict = analysis.build_bitdiff_dictionary(packets)

In [26]:
b1 = [key for key in cracked_bits_dict.keys() if len(key.split(':')) == 1]
b1.sort()
b1

['116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '134',
 '135',
 '3',
 '38',
 '39',
 '4',
 '42',
 '43',
 '44',
 '45',
 '5',
 '6',
 '66',
 '67',
 '7',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92']

In [19]:
cracked_bits_dict['86']

33755

In [25]:
# HERE HERE HERE
# This is the algo that is optimizing the dictionary to infer
# single bit diffs from larger diffs

for key in cracked_bits_dict.keys():
    key_values = key.split(':')
    if len(key_values) != 4:
        continue
    dc = cracked_bits_dict[key]
    remaining_bits = []
    #print "-" * 80
    #print "Working on %s -> 0x%04x" % (key, dc)
    for b in key.split(':'):
        if b in b1:
            dc = dc ^ cracked_bits_dict[b]
            #print "dc now 0x%04x" % dc
        else:
            remaining_bits.append(b)
        
    if len(remaining_bits) == 0:
        #print "removing key"
        del cracked_bits_dict[key]
    elif len(remaining_bits) == 1:
        del cracked_bits_dict[key]
        print "discovered xor for bit %s" % remaining_bits[0]
        cracked_bits_dict[remaining_bits[0]] = dc
    else:
        print "missing xors for bits %s" % remaining_bits
            

missing xors for bits ['116', '117']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['116', '117']
missing xors for bits ['116', '117']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['84', '116']
missing xors for bits ['116', '117']
missing xors for bits ['84', '117']
missing xors for bits ['116', '117']
missing xors for bits [

In [ ]:
xors = [cracked_bits_dict[key] for key in cracked_bits_dict.keys() if len(key.split(':')) == 1]

In [ ]:
v = ["0x%04x" % x for x in xors]
pd.Series(v).value_counts()